# import

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

In [22]:
coupang = pd.read_csv('../../data/coupang_filtering.csv', index_col=0)
display(coupang.head())

year = datetime.today().year
month = datetime.today().month
day = datetime.today().day
naver_df = pd.read_csv(f'../../data/{str(year)}-{str(month)}-{str(day)}edited_navertrend.csv', index_col=0)
display(naver_df.head())

,index,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
0,0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11454,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골
1,1,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15941,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골
2,2,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49899,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎",전골
3,3,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,37082,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골
4,4,1,6310053892,13092768757,80353833768,마이셰프 EBS 최요비 산더미 소고기 콩불 2인분,"마이셰프 EBS 최요비 산더미 소고기 콩불 2인분, 810g, 1개",14900,20%,11900,"(100g당 1,469원)",4.5,3072,0,"소고기, 콩나물, 대파, 양파, 깻잎, 소스로 구성되어 있습니다.","소고기,콩나물,대파,양파,깻잎,소스",한식


,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트,가중치
0,밀키트,참한 전복,밀키트,밀키트,밀키트,밀키트,캠핑요리추천,떡볶이,1.000
1,떡볶이,리조또 밀키트,낙곱새,동남아식 밀키트 탄탄면,스테이크 밀키트,감바스 밀키트,캠핑음식추천,밀키트,0.995
2,낙곱새,프레시지 리조또,알탕,파스타 밀키트,자미곱,감바스,밀키트,쌀떡볶이,0.990
3,알탕,옹기전 팥죽,캠핑 밀키트,수제 비 밀키트,목살 밀키트,애슐리 밀키트,찜닭밀키트,국물떡볶이,0.985
4,캠핑 밀키트,빠에야 밀키트,해신탕,비빔국수 밀키트,군고기 밀키트,돈이 곱창 참숯 초벌 냄새없는 곱창 볶음 순대볶음 간단 캠핑 요리 밀키트 돼지 곱창...,갈비찜오리지널맛,사과떡볶이,0.980


# coupang

## 품절 상품 제외

In [28]:
# allergy_num = '4'
# coupang = coupang[(coupang.품절여부 == 0)&(~coupang['알레르기'].str.contains(allergy_num))]
coupang = coupang.loc[coupang['품절여부'] == 0]
coupang.reset_index(inplace=True)
coupang.head(3)

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11454,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골
1,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15941,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골
2,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49899,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎",전골


## 알레르기 추가
알류 (가금류)
우유
메밀
땅콩
대두
밀
잣
호두
게
새우
오징어
고등어
조개류 (굴, 전복, 홍합)
복숭아
토마토
닭고기
돼지고기
쇠고기
아황산류


In [13]:
allergy_dict = {'갑각류' : ['새우', '꽃게', '랍스터', '가재','대하','쉬림프','감바스'],
                '견과류' : ['밤', '호두', '땅콩', '아몬드', '잣', '은행', '헤이즐넛', '크랜베리','캐슈넛','참깨','참기름','들기름','들깨'],
                '난류' : ['달걀', '계란', '오코노미야끼', '수란','알 가열제품','반숙란','지단'],
                '메밀' : ['메밀'],
                '밀가루' : ['바게트','통돈까스','빵','밀가루','또띠아','부침개','수제비','칼국수','라면','우동','유탕면','탄탄면','중화면','중화 면','건면','생면','파스타','스파게티','숙면','소면','쫄면','만두','밀떡','생중면','사리'],
                '우유' : ['우유','치즈','버터','마가린','요거트','요구르트','빠네','투움바','까르보나라','크림','리조또'],
                '연체동물류' : ['조개', '홍합', '굴', '가리비','바지락','전복','꼬막','해산물','해물','문어', '오징어','낙지','연포탕','해신탕','낙곱새','성게','우렁살','봉골레','관자'],
                '콩' : ['두부', '대두', '간장', '된장','콩나물','콩기름'],
                '닭' : ['닭','치킨'],
                '돼지' : ['돼지','통돈까스','부대찌개','돔베','햄','코테기노','돈마호크','베이컨','탄탄면','볼로네제','비엔나','족발','보쌈','마제소바','목살','양장피','통삼겹','포크'],
                '복숭아' : ['복숭아'],
                '소' : ['소고기', '쇠고기','소 스지','샤브샤브','도가니','양지','육개장','우삼겹','찹스테이크','볼로네제','비프','비엔나'],
                '토마토' : ['토마토','볼로네제'],
                '생선류' : ['굴비', '조기', '고등어', '갈치', '꽁치', '전어', '명태', '노가리', '메기', '동태','곤이','아귀','아구','코다리','대구','복지리','우럭','가자미','도다리','해물탕','어묵']}


In [70]:
def allergy(allergy_dict, dataframe, column, column2):
    allergy_list = []
    for idx, product in enumerate(dataframe[column]):
        sublist = []
        for key in list(allergy_dict.keys()):
            for i in allergy_dict[key]:
                if i in product:
                    sublist.append(key)
                if i in dataframe.loc[idx,column2]:
                    sublist.append(key)
        sublist = list(set(sublist))
        sublist = ', '.join(sublist)
        allergy_list.append(sublist)
    return allergy_list

In [71]:
allergy_list = allergy(allergy_dict, coupang, '상품', '구성정보')
allergy_series = pd.Series(allergy_list)
coupang['알레르기'] = allergy_series
# coupang.to_csv('../../data/coupang_filtering.csv')
coupang.head(2)

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,알레르기,weighted_rating
0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11454,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골,"밀가루, 콩, 소",4.498910
1,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15941,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골,"밀가루, 콩, 소",4.499207


## 쿠팡 별점, 리뷰수 가중치 
https://data-newbie.tistory.com/817

In [62]:
def weighted_rating(v,m,R,C):
    '''
    Calculate the weighted rating
    
    Args:
    v -> average rating for each item (float)
    m -> minimum votes required to be classified as popular (float)
    R -> average rating for the item (pd.Series)
    C -> average rating for the whole dataset (pd.Series)
    # ( (각 리뷰수 / (각 리뷰수+최소리뷰수)) * 각 별점 ) + ( (최소리뷰수/(각 리뷰수+최소리뷰수)) * 평균별점 )
    
    Returns:
    pd.Series
    '''
    return ( (v / (v + m)) * R) + ( (m / (v + m)) * C )

In [63]:
# calcuate input parameters
C = np.mean(coupang['별점'])
m = np.percentile(coupang['리뷰수'], 70)
vote_count = coupang[coupang['리뷰수'] >= m]
R = coupang['별점']
v = coupang['리뷰수']

coupang['weighted_rating'] = weighted_rating(v,m,R,C)
coupang.to_csv('../../data/coupang_filtering.csv')
# coupang.sort_values('weighted_rating')

,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,알레르기,weighted_rating
42,1,6011661740,10907721918,78187554269,속초 동태탕 (냉동),"속초 동태탕 (냉동), 500g, 2팩",14000,0,14000,"(100g당 1,400원)",4.0,644,0,"명태, 명태알, 명태 곤이, 주름 미더덕, 얼큰 분말 스프로 구성되어 있습니다.","명태,명태알,명태곤이",생선탕,생선류,4.205186
57,1,1183877130,2164259048,70162428267,마이셰프 냄비가득 고기 김치찌개,"마이셰프 냄비가득 고기 김치찌개, 575g, 1개",10900,0,10900,"(100g당 1,896원)",4.0,577,0,"라면사리, 소고기, 배추김치, 육수, 양파, 대파로 구성되어 있습니다.","라면사리,소고기,배추김치,육수,양파,대파",찌개/짜글이,"밀가루, 소",4.218041
51,1,6465462490,14090734122,81337922656,프레시지 통낙지 해신탕 (냉동),"프레시지 통낙지 해신탕 (냉동), 835g, 1개",19900,20%,15920,"(100g당 1,907원)",4.0,544,0,"야채(배추, 양파, 대파, 표고버섯), 소스, 닭고기 포장육, 수산물가공품으로 구성...","야채,배추,양파,대파,표고버섯,소스,닭고기 포장육",해물탕,"닭, 연체동물류",4.224984
48,1,6764025028,15859796330,83069850181,프레시지 얼큰한 동태탕 (냉동),"프레시지 얼큰한 동태탕 (냉동), 785g, 1개",14900,0,14900,"(100g당 1,898원)",4.0,139,0,"기타 수산물가공품(명태, 대구곤이, 홍합살), 채소(무, 대파, 청양고추), 소스로...","명태,대구곤이,홍합살,채소,무,대파,청양고추,소스",생선탕,"생선류, 연체동물류",4.369296
104,4,6638342207,15165313309,82386708499,바다자리 새우 해물 감바스 밀키트 1~2인분 240g,"바다자리 새우 해물 감바스 밀키트 1~2인분 240g, 1세트",16100,17%,13300,NaN,4.0,136,0,"바지락, 새우살, 오징어, 건고추, 양파, 마늘, 올리브오일, 허브솔트로 구성되어 ...","바지락,새우살,오징어,건고추,양파,마늘,올리브오일,허브솔트",감바스,"갑각류, 연체동물류",4.371059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,4,1600874220,2734537090,70724641036,곰곰 감바스 알 아히요,"곰곰 감바스 알 아히요, 492g, 1세트",13990,0,13990,"(100g당 2,843원)",5.0,8039,0,"중새우, 감자수제비, 바케트빵, 새송이버섯, 파마산, 치즈, 베트남, 고추, 마늘,...","중새우,감자수제비,바케트빵,새송이버섯,파마산,치즈,베트남,고추,마늘,브로콜리,올리브유",감바스,"갑각류, 밀가루, 우유",4.969615
97,4,156649272,451065872,4117038563,프레시지 쉬림프 로제 파스타,"프레시지 쉬림프 로제 파스타, 562g, 1개",11900,16%,9900,"(100g당 1,762원)",5.0,11718,0,"수산물가공품(새우 함유), 양파, 소스, 건면 등으로 구성되어 있습니다.","양파,소스,건면",파스타,"갑각류, 밀가루",4.978770
127,6,293473507,926572569,5302426608,마이셰프 매콤 소고기 고추 잡채 꽃빵,"마이셰프 매콤 소고기 고추 잡채 꽃빵, 570g, 1개",13900,0,13900,"(100g당 2,439원)",5.0,12531,0,"1. 꽃빵, 소고기, 손질한 양파와 파프리카, 죽순, 표고버섯, 피망, 청양고추,...","꽃빵,소고기,파프리카,죽순,표고버섯,피망,청양고추,고추 기름",볶음,"밀가루, 소",4.980095
87,3,2212937714,3767825253,71752894115,곰곰 골든비프 찹스테이크 밀키트,"곰곰 골든비프 찹스테이크 밀키트, 571g, 1개",16990,5%,15990,"(100g당 2,800원)",5.0,13389,0,"쇠고기, 양파, 홍피망, 청피망, 쥬키니호박, 총알새송이버섯, 방울토마토, 마늘종,...","쇠고기,양파,홍피망,청피망,쥬키니호박,총알새송이버섯,방울토마토,마늘종,단호박",스테이크,"소, 토마토",4.981326


In [ ]:
print(len(coupang[coupang.리뷰수 == 0]))
coupang[coupang.리뷰수 == 0].head()

43


,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,weighted_rating
90,1,6919151669,16702268903,83884772135,해물래 푸짐한 해물뚝배기 탕,"해물래 푸짐한 해물뚝배기 탕, 1개, 540g",13890,0,13890,"(100g당 2,572원)",0.0,0,0,"오징어, 주꾸미, 새우, 홍합, 바지락, 고추, 대파, 무, 소스로 구성되어 있습니다.","오징어,주꾸미,새우,홍합,바지락,고추,대파,무,소스",해물탕,3.883901
100,1,7033239461,17364429594,84534356834,면사랑 김치전골우동 밀키트 2인분 (냉동),"면사랑 김치전골우동 밀키트 2인분 (냉동), 808g, 1개",11900,0,11900,"(100g당 1,473원)",0.0,0,1,"면, 우삼겹, 볶음김치, 유부, 어묵, 대파, 양파 등이 각 2개씩 총 2인분으로 ...","면,우삼겹,볶음김치,유부,어묵,대파",전골,3.883901
101,1,7033239068,17364427023,84534354874,면사랑 사골떡만두국 밀키트 2인분 (냉동),"면사랑 사골떡만두국 밀키트 2인분 (냉동), 1개, 551g",9900,0,9900,"(100g당 1,797원)",0.0,0,1,"떡, 두 가지 만두, 옛날 고명 지단, 냉동 대파, 조미김 등이 각 2개씩 총 2인...","떡,냉동 대파",면,3.883901
103,1,7006320086,17201669580,84373653564,마이셰프 시원한 동태탕 (냉동),"마이셰프 시원한 동태탕 (냉동), 1개, 600g",11000,0,11000,"(100g당 1,833원)",0.0,0,1,"소스, 무, 대파, 명태, 명태곤이, 명란으로 구성되어 있습니다.","소스,무,대파,명태,명태곤이,명란",생선탕,3.883901
104,1,7006319842,17201668593,84373652653,마이셰프 얼큰한 꽃게탕 (냉동),"마이셰프 얼큰한 꽃게탕 (냉동), 1개, 500g",11000,0,11000,"(100g당 2,200원)",0.0,0,1,"소스, 무, 대파, 꽃게로 구성되어 있습니다.","소스,무,대파,꽃게",해물탕,3.883901


# 네이버 
> 가중치 추가

In [64]:
# 가중치 추가 
naver_len = len(naver_df)
naver_df['가중치'] = [naver_weight/naver_len for naver_weight in range(naver_len,0, -1)]

# 가장 많이 나오는 밀키트 단어 삭제
naver_df.replace('밀키트','', regex=True, inplace=True)

# 쿠팡 상세카테고리와 맞게 일치시킴
from CoupangCategoryName import category_name
naver_df['전체_카테고리'] = naver_df['전체'].apply(lambda x : category_name(x))
naver_df['밥/죽_카테고리'] = naver_df['밥/죽'].apply(lambda x : category_name(x))
naver_df['찌개/국_카테고리'] = naver_df['찌개/국'].apply(lambda x : category_name(x))
naver_df['면/파스타_카테고리'] = naver_df['면/파스타'].apply(lambda x : category_name(x))
naver_df['구이_카테고리'] = naver_df['구이'].apply(lambda x : category_name(x))
naver_df['볶음/튀김_카테고리'] = naver_df['볶음/튀김'].apply(lambda x : category_name(x))
naver_df['조림/찜_카테고리'] = naver_df['조림/찜'].apply(lambda x : category_name(x))
naver_df['간식/디저트_카테고리'] = naver_df['간식/디저트'].apply(lambda x : category_name(x))
naver_df

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트,가중치,전체_카테고리,밥/죽_카테고리,찌개/국_카테고리,면/파스타_카테고리,구이_카테고리,볶음/튀김_카테고리,조림/찜_카테고리,간식/디저트_카테고리
0,,참한 전복,,,,,캠핑요리추천,떡볶이,1.000,기타,기타,기타,기타,기타,기타,기타,떡볶이
1,떡볶이,리조또,낙곱새,동남아식 탄탄면,스테이크,감바스,캠핑음식추천,,0.995,떡볶이,리조또,기타,면,스테이크,감바스,기타,기타
2,낙곱새,프레시지 리조또,알탕,파스타,자미곱,감바스,,쌀떡볶이,0.990,기타,리조또,알탕,파스타,기타,감바스,기타,떡볶이
3,알탕,옹기전 팥죽,캠핑,수제 비,목살,애슐리,찜닭,국물떡볶이,0.985,알탕,기타,기타,기타,기타,기타,닭,떡볶이
4,캠핑,빠에야,해신탕,비빔국수,군고기,돈이 곱창 참숯 초벌 냄새없는 곱창 볶음 순대볶음 간단 캠핑 요리 돼지 곱창 볶음...,갈비찜오리지널맛,사과떡볶이,0.980,기타,기타,해물탕,국수,기타,볶음,찜,떡볶이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,곱도리탕 캠핑용,버섯 야채죽,저렴한 캠핑,코델리아 파스타,가니쉬,윌터 감바스,전자레인지,진짜떡볶이즉석,0.025,기타,기타,기타,파스타,기타,감바스,기타,떡볶이
196,먹보 소고기 국밥,부방대마켓,조방 낙지 낙곱새,분짜,주왕산 돼지갈비,짐승 파스타,짚신,즉석진짜떡볶이,0.020,국,기타,기타,기타,기타,파스타,기타,떡볶이
197,마라샹궈,부타동,라라관,일산 닭 칼국수,문현 한우 곱창전골,홈파티,찜갈비,요리놀이터,0.015,기타,기타,기타,닭,전골,기타,찜,기타
198,모듬 튀김,분말 죽,개미집 낙곱새,골뱅이,빕스 갈릭 버터 스테이크,생생 닭갈비 캠핑 닭갈비 춘천 생생 닭갈비,프레시지,즉석떡볶이,0.010,기타,기타,기타,기타,스테이크,닭,기타,떡볶이


In [65]:
# 네이버트렌드 카테고리 : 상세카테고리
dict1 = {'밥/죽' : ['리조또', '한식'],
         '찌개/국' : ['전골', '생선탕', '찌개/짜글이', '알탕', '해물탕', '국', '마라탕'],
         '면/파스타' : ['면', '국수', '파스타'],
         '구이' : ['구이', '스테이크'],
         '볶음/튀김' : ['볶음', '감바스', '순대'],
         '조림/찜' : ['찜', '마파두부'],
         '간식/디저트' : ['샌드위치', '족발'],
         '전체' : ['씨푸드', '닭', '떡볶이', '무침', '쌈', '카레']}

# 밀키트 추천 시스템 구현
> CountVectorizer & TfidfVectorizer #민규님

> 리뷰순을 가중치로 바꾼다면?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def coupang_naver_categories(category):
    
    if category in dict1['밥/죽'] : 
        naver_value = naver_df.loc[naver_df['밥/죽_카테고리'] == category]
        naver_value = naver_value['밥/죽'].values
        naver_value = naver_value.tolist()
    elif category in dict1['찌개/국'] : 
        naver_value = naver_df.loc[naver_df['찌개/국_카테고리'] == category]
        naver_value = naver_value['찌개/국'].values
        naver_value = naver_value.tolist()
    elif category in dict1['구이'] :
        naver_value = naver_df.loc[naver_df['구이_카테고리'] == category]
        naver_value = naver_value['구이'].values
        naver_value = naver_value.tolist()
    elif category in dict1['면/파스타'] : 
        naver_value = naver_df.loc[naver_df['면/파스타_카테고리'] == category]
        naver_value = naver_value['면/파스타'].values
        naver_value = naver_value.tolist()
    elif category in dict1['볶음/튀김'] : 
        naver_value = naver_df.loc[naver_df['볶음/튀김_카테고리'] == category]
        naver_value = naver_value['볶음/튀김'].values
        naver_value = naver_value.tolist()
    elif category in dict1['조림/찜'] : 
        naver_value = naver_df.loc[naver_df['조림/찜_카테고리'] == category]
        naver_value = naver_value['조림/찜'].values
        naver_value = naver_value.tolist()
    elif category in dict1['간식/디저트'] : 
        naver_value = naver_df.loc[naver_df['간식/디저트_카테고리'] == category]
        naver_value = naver_value['간식/디저트'].values
        naver_value = naver_value.tolist()
    elif category in dict1['전체'] : 
        naver_value = naver_df.loc[naver_df['전체_카테고리'] == category]
        naver_value = naver_value['전체'].values
        naver_value = naver_value.tolist()
    else:
        pass
    
    product_list = naver_value

    return category, product_list


def cv_commend_system(category, product_list):
    extracted_productid = []
    try:
        # 네이버 트렌드 상위 5개 키워드와 유사한 쿠팡 밀키트 추출
        for num in range(5):
            # TfidfVectorizer과 코사인 유사도를 이용한 추천시스템 
            docs = coupang.loc[coupang['상세카테고리'] == category].상품.values
            docs = np.append(docs,product_list[num])
            vect = CountVectorizer()
            countvect = vect.fit_transform(docs)
            countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
            
            # 위의 Data Frame 형태의 유사도를 계산 
            cosine_matrix = cosine_similarity(countvect_df, countvect_df)
            productid = {}
            for i, c in enumerate(coupang.loc[coupang['상세카테고리'] == category].상품): productid[i] = c
            productid[len(productid)] = product_list[num]
            
            product = {}
            for i, c in productid.items(): product[c] = i
            
            idx = product[product_list[num]]
            sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
            sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)  
            
        
            exproduct = productid[sim_scores[0][0]]
            if exproduct not in extracted_productid :
                if sim_scores[0][1] <= 0.01 :
                    continue
                extracted_productid.append(exproduct)
                print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[0][1])
            
            else:
                exproduct = productid[sim_scores[1][0]]
                if exproduct not in extracted_productid :
                    if sim_scores[1][1] <= 0.01 :
                        continue
                    extracted_productid.append(exproduct)
                    print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[1][1])
                
                else :
                    exproduct = productid[sim_scores[2][0]]
                    if exproduct not in extracted_productid :
                        if sim_scores[2][1] <= 0.01 :
                            continue
                        extracted_productid.append(exproduct)
                        print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[2][1])
                        
                    else :
                        exproduct = productid[sim_scores[3][0]]
                        if exproduct not in extracted_productid :
                            if sim_scores[3][1] <= 0.01 :
                                continue
                            extracted_productid.append(exproduct)
                            print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[3][1])
                            
                        else :
                            exproduct = productid[sim_scores[4][0]]
                            if exproduct not in extracted_productid :
                                if sim_scores[4][1] <= 0.01 :
                                    continue
                                extracted_productid.append(exproduct)
                                print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[4][1])
                    
        if len(extracted_productid) != 5 :
            raise Exception()

    # 선택한 카테고리가 네이버 트렌드 데이터에 포함되어있지 않을 때
    except :
        try:
            coupang_sorted = coupang.loc[coupang['상세카테고리'] == category]
            coupang_sorted = coupang_sorted.astype({'리뷰수' : int})
            coupang_sorted = coupang_sorted.sort_values('리뷰수', ascending = False)
            if len(extracted_productid) == 4: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                        
            elif len(extracted_productid) == 3: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
            
            elif len(extracted_productid) == 2: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                    
            elif len(extracted_productid) == 1:
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                        
            elif len(extracted_productid) == 0:
                for number in range(5):
                    exproduct = coupang_sorted.iloc[number].상품
                    extracted_productid.append(exproduct)
                    print(exproduct)
                
                
        except:
            pass
    
    return extracted_productid
        
    
def tfidf_commend_system(category, product_list):
    extracted_productid = []
    try:
        # 네이버 트렌드 상위 5개 키워드와 유사한 쿠팡 밀키트 추출
        for num in range(5):
            # TfidfVectorizer과 코사인 유사도를 이용한 추천시스템 
            docs = coupang.loc[coupang['상세카테고리'] == category].상품.values
            docs = np.append(docs,product_list[num])
            vect = TfidfVectorizer()
            tfidfvect = vect.fit_transform(docs)
            tfidfvect_df = pd.DataFrame(tfidfvect.toarray(), columns = sorted(vect.vocabulary_))
            
            # 위의 Data Frame 형태의 유사도를 계산 
            cosine_matrix = cosine_similarity(tfidfvect_df, tfidfvect_df)
            productid = {}
            for i, c in enumerate(coupang.loc[coupang['상세카테고리'] == category].상품): productid[i] = c
            productid[len(productid)] = product_list[num]
            
            product = {}
            for i, c in productid.items(): product[c] = i
            
            idx = product[product_list[num]]
            sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
            sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)  
            
        
            exproduct = productid[sim_scores[0][0]]
            if exproduct not in extracted_productid :
                if sim_scores[0][1] <= 0.01 :
                    continue
                extracted_productid.append(exproduct)
                print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[0][1])
            
            else:
                exproduct = productid[sim_scores[1][0]]
                if exproduct not in extracted_productid :
                    if sim_scores[1][1] <= 0.01 :
                        continue
                    extracted_productid.append(exproduct)
                    print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[1][1])
                
                else :
                    exproduct = productid[sim_scores[2][0]]
                    if exproduct not in extracted_productid :
                        if sim_scores[2][1] <= 0.01 :
                            continue
                        extracted_productid.append(exproduct)
                        print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[2][1])
                        
                    else :
                        exproduct = productid[sim_scores[3][0]]
                        if exproduct not in extracted_productid :
                            if sim_scores[3][1] <= 0.01 :
                                continue
                            extracted_productid.append(exproduct)
                            print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[3][1])
                            
                        else :
                            exproduct = productid[sim_scores[4][0]]
                            if exproduct not in extracted_productid :
                                if sim_scores[4][1] <= 0.01 :
                                    continue
                                extracted_productid.append(exproduct)
                                print('현재 트렌드 -', docs[-1], '과 유사한 상품은', exproduct, sim_scores[4][1])
                    
        if len(extracted_productid) != 5 :
            raise Exception()

    # 선택한 카테고리가 네이버 트렌드 데이터에 포함되어있지 않을 때
    except :
        try:
            coupang_sorted = coupang.loc[coupang['상세카테고리'] == category]
            coupang_sorted = coupang_sorted.astype({'리뷰수' : int})
            coupang_sorted = coupang_sorted.sort_values('리뷰수', ascending = False)
            if len(extracted_productid) == 4: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                        
            elif len(extracted_productid) == 3: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
            
            elif len(extracted_productid) == 2: 
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                    
            elif len(extracted_productid) == 1:
                for number in range(7):
                    exproduct = coupang_sorted.iloc[number].상품
                    if exproduct in extracted_productid :
                        pass
                    else :
                        extracted_productid.append(exproduct)
                        print(exproduct)
                        if len(extracted_productid) == 5:
                            break
                        
            elif len(extracted_productid) == 0:
                for number in range(5):
                    exproduct = coupang_sorted.iloc[number].상품
                    extracted_productid.append(exproduct)
                    print(exproduct)

        except:
            pass
    
    return extracted_productid

In [ ]:
category, product_list = coupang_naver_categories('생선탕')
tfidf_commend = tfidf_commend_system(category, product_list)

1. 네이버 트렌드 새로 데이터 추출해서, 띄어쓰기 사전 보충 O

2. 네이버 트렌드 데이터로 추천시스템 돌려보기

3. 조건식을 통해 유사도가 특정 수치 이하면, 그냥 패스시켜주고 그 카테고리인 애들 보여주는 정도? 여러 개중 리뷰수 기준?

4. 태현님 알레르기파일까지


1. 네이버트렌드를 이용해 추출한 쿠팡 밀키트 중 품절 상품 제외

2. 알레르기 재료 필터링(상품명 , 구성정보로부터) or 명시, 요기요

3. 추출한 쿠팡 밀키트 - 만개의 레시피 쿠팡 데이터 연결

# w2v 밀키트 추천시스템 구현 
> fail 원하는 결과가 안 나옴...

In [ ]:
#상세카테고리와 네이버트렌드 카테고리가 일치하는 트렌드 리스트 반환
def coupang_naver_categories(category):
    if category in dict1['찌개/국']:
        naver_value = naver_df[['찌개/국', '가중치']].values
    elif category in dict1['면/파스타']:
        naver_value = naver_df[['면/파스타', '가중치']].values
    elif category in dict1['구이']:
        naver_value = naver_df[['구이', '가중치']].values
    elif category in dict1['볶음/튀김']:
        naver_value = naver_df[['볶음/튀김', '가중치']].values
    elif category in dict1['조림/찜']:
        naver_value = naver_df[['조림/찜', '가중치']].values
    else:
        naver_value = naver_df[['전체', '가중치']].values
    naver_value = naver_value.tolist()
    return naver_value

In [ ]:
# category = '전골'
# trendlist = []
# for trend in coupang_naver_categories(category):
#     if category_name(trend[0]) == category:
#         trendlist.append(trend)       
# trendlist

[['충무로 한우 곱창전골', 0.975],
 ['만두 전골', 0.97],
 ['곱창전골 밀키트', 0.965],
 ['곱창전골', 0.96],
 ['더원푸드 시래기 곱창전골 밀키트', 0.945],
 ['밀푀유 나베 밀키트', 0.93],
 ['더원푸드 시래기 전골 밀키트', 0.925],
 ['곱창전골 한우 소곱창전골', 0.915],
 ['해물 전골 캠핑 밀키트', 0.9],
 ['밀푀유 나베', 0.89],
 ['밀키트 만두 전골', 0.885],
 ['소고기 전골 밀키트', 0.855],
 ['문현 한우 곱창전골', 0.805],
 ['호랑이 굴대창전골', 0.79],
 ['기승 전골', 0.78],
 ['간편하게 즐기는 곱창전골', 0.755],
 ['모츠 나베', 0.71],
 ['대창 전골', 0.655],
 ['한우 곱창전골', 0.65],
 ['형제 소곱창전골', 0.645],
 ['스키야키 밀키트', 0.635],
 ['청어람 곱창전골', 0.62],
 ['프레시지 밀푀유 나베', 0.605],
 ['모츠 나베 밀키트', 0.6],
 ['버섯전골', 0.585],
 ['스키야키 2인 분', 0.58],
 ['기승 전골 밀키트', 0.575],
 ['캠핑 전골 밀키트', 0.57],
 ['전골', 0.485],
 ['캠핑 곱창전골 밀키트', 0.48],
 ['돼지 곱창전골', 0.455],
 ['곱창 지존 곱창전골', 0.435],
 ['곱창전골 캠핑 밀키트', 0.43],
 ['만두 전골 밀키트', 0.41],
 ['스지 전골', 0.38],
 ['전골 밀키트', 0.335],
 ['호랑이 굴대창전골 밀키트', 0.325],
 ['대한 곱창전골', 0.305],
 ['스키야키', 0.3],
 ['알 곱창전골', 0.23],
 ['아롱사태 전골', 0.19],
 ['한우리 국수 전골', 0.13],
 ['한우 곱창전골 밀키트', 0.085],
 ['소곱창전골 밀키트', 0.075],
 ['한구 곱창전골', 0.025]]

> 파라미터 설명
- https://hoonzi-text.tistory.com/2
- https://velog.io/@mstar228/GensimWord2Vec

In [ ]:
# for key in trendlist:
#     sentence = [[key[0]]]
#     for cg in coupang[coupang['상세카테고리']==category].상품.values:
#         sentence.append([cg])
#     embedding_model = Word2Vec(sentence, vector_size=300, window=3, min_count=1, sg=1)
#     a = embedding_model.wv.most_similar(positive=[key], topn=3)
#     print(a)

In [ ]:
# category = '떡볶이'
# sentence = [['미미네 눈꽃 국물 떡볶이 570 g']]
# for cg in coupang[coupang['상세카테고리']==category].상품.values:
#     sentence.append([cg])
# print(sentence)
# embedding_model = Word2Vec(sentence, vector_size=300, window=3, min_count=1, sg=1)
# embedding_model.wv.most_similar(positive=['미미네 눈꽃 국물 떡볶이 570 g'], topn=3)

[['미미네 눈꽃 국물 떡볶이 570 g'], ['프레시지 더큰 호로록 우삼겹 떡볶이'], ['마이셰프 카카오프렌즈 튜브 우삼겹 떡볶이'], ['마이셰프 TREASURE HUNTER 우정분식 아찔 떡볶잉 2인분 (냉동)'], ['복선당 단호박 대창 떡볶이 (냉동)'], ['프레시지 박막례 치즈 떡볶이 (냉동)'], ['쉐푸드 곱창 고구마 떡볶이 밀키트 (냉동)']]


[('복선당 단호박 대창 떡볶이 (냉동)', 0.1341700702905655),
 ('마이셰프 TREASURE HUNTER 우정분식 아찔 떡볶잉 2인분 (냉동)', 0.07046209275722504),
 ('쉐푸드 곱창 고구마 떡볶이 밀키트 (냉동)', -0.014022953808307648)]

In [ ]:
# category = '떡볶이'
# sentence = [['돈마호크 카츠']]
# for cg in coupang.상품.values:
#     sentence.append([cg])
# embedding_model = Word2Vec(sentence, vector_size=300, window=3, min_count=1, sg=1)
# embedding_model.wv.most_similar(positive=['돈마호크 카츠'], topn=3)

[('쉐푸드 동태알탕 밀키트 (냉동)', 0.18393783271312714),
 ('바다자리 새우 해물 감바스 밀키트 1~2인분 240g', 0.14130397140979767),
 ('바다자리 새꼬막살 무침용 씨키트 1~2인분 150g', 0.13561250269412994)]

In [ ]:
# sentence = [['미미네 눈꽃 국물 떡볶이 570 g']]
# for cg in coupang.상품.values:
#     sentence.append([cg])
# embedding_model = Word2Vec(sentence, vector_size=300, window=5, min_count=1, sg=1)
# embedding_model.wv.most_similar(positive=['미미네 눈꽃 국물 떡볶이 570 g'], topn=3)

[('쉐푸드 동태알탕 밀키트 (냉동)', 0.18393783271312714),
 ('바다자리 새우 해물 감바스 밀키트 1~2인분 240g', 0.14130397140979767),
 ('바다자리 새꼬막살 무침용 씨키트 1~2인분 150g', 0.13561250269412994)]

In [ ]:
# embedding_model.wv.most_similar(positive=['마이셰프 TREASURE HUNTER 우정분식 아찔 떡볶잉 2인분 (냉동)'], topn=3)

[('쉐푸드 곱창 고구마 떡볶이 밀키트 (냉동)', 0.08134971559047699),
 ('미미네 눈꽃 국물 떡볶이 570 g', 0.07046209275722504),
 ('복선당 단호박 대창 떡볶이 (냉동)', 0.05005084350705147)]

In [ ]:
# sentence = [['동태탕 밀키트']]
# for cg in coupang[coupang['상세카테고리']==category].상품.values:
#     sentence.append([cg])
# embedding_model = Word2Vec(sentence, vector_size=20, window=5, min_count=1, workers=4, sg=1)
# a = embedding_model.wv.most_similar(positive=['동태탕 밀키트'], topn=10)
# a

[('프레시지 시원한 대구탕 (냉동)', 0.45488056540489197),
 ('마이셰프 시원한 동태탕 (냉동)', 0.43684545159339905),
 ('바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채', 0.38747790455818176),
 ('펀쿡 복지리탕 (냉동)', 0.33371350169181824),
 ('본갯마을 동태탕 (냉동)', 0.3148423433303833),
 ('99프레시밀키트 보리새우버섯 매운탕 밀키트', 0.1816919595003128),
 ('바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채', 0.18036746978759766),
 ('속초 동태탕 (냉동)', 0.1357535421848297),
 ('미소프레쉬 6컵 푸짐한 대구탕', 0.1289648711681366),
 ('바다자리 겨울엔 생물 물메기 매운탕 씨키트 2~3인분 800g + 야채', 0.08396223932504654)]

In [ ]:
# coupang[coupang['상세카테고리']==category].상품.values

array(['미소프레쉬 6컵 푸짐한 대구탕', '바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채',
       '속초 동태탕 (냉동)', '바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채',
       '바다자리 겨울엔 생물 물메기 맑은탕 씨키트 2~3인분 700g + 야채',
       '바다자리 생물 아귀 맑은탕 씨키트 2~3인분 900g + 야채', '속초 코다리맑음탕 (냉동)',
       '99프레시밀키트 보리새우버섯 매운탕 밀키트', '어부담 백조기 굴비매운탕', '프레시지 얼큰한 동태탕 (냉동)',
       '바다자리 겨울엔 생물 물메기 매운탕 씨키트 2~3인분 800g + 야채', '아라보감 산지직송 우럭매운탕용 씨키트',
       '바다자리 겨울엔 생물 대구 매운탕 씨키트 2~3인분 800g + 야채', '프레시지 시원한 대구탕 (냉동)',
       '펀쿡 복지리탕 (냉동)', 'the 푸짐한 탕선생 한마리 동태탕 (냉동)', '아라보감 산지직송 우럭지리탕용 씨키트',
       '마이셰프 시원한 동태탕 (냉동)', '본갯마을 동태탕 (냉동)'], dtype=object)

In [ ]:
# sentence = [['매운탕']]
# for cg in coupang[coupang['상세카테고리']==category].상품.values:
#     sentence.append([cg])
# embedding_model = Word2Vec(sentence, vector_size=20, window=5, min_count=1, workers=4, sg=1)
# a = embedding_model.wv.most_similar(positive=['매운탕'], topn=10)
# a

[('프레시지 시원한 대구탕 (냉동)', 0.45488056540489197),
 ('마이셰프 시원한 동태탕 (냉동)', 0.43684545159339905),
 ('바다자리 겨울엔 생물 대구 맑은탕 씨키트 2~3인분 700g + 야채', 0.38747790455818176),
 ('펀쿡 복지리탕 (냉동)', 0.33371350169181824),
 ('본갯마을 동태탕 (냉동)', 0.3148423433303833),
 ('99프레시밀키트 보리새우버섯 매운탕 밀키트', 0.1816919595003128),
 ('바다자리 생물 아귀 매운탕 씨키트 2~3인분 1kg + 야채', 0.18036746978759766),
 ('속초 동태탕 (냉동)', 0.1357535421848297),
 ('미소프레쉬 6컵 푸짐한 대구탕', 0.1289648711681366),
 ('바다자리 겨울엔 생물 물메기 매운탕 씨키트 2~3인분 800g + 야채', 0.08396223932504654)]